# IMPORT

In [1]:
# Geometry packages
import geocoder
# Web Scraping
import requests
from bs4 import BeautifulSoup
# Data Analysis
import pandas as pd
import numpy as np
# Data Visualization

# Read Table into DataFrame

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df = dfs[0]
# remove Not assigned Borough
df = df[df['Borough'] != "Not assigned"]
# assign Borough tco NaN Neighbourhood
for i in range(len(df)):
    if df.iloc[i,2] == 'Not assigned':
        df.iloc[i,2] = df.iloc[i,1]
# combine areas with the same postal code
functions = {'Borough': 'sum', 'Neighbourhood': lambda column: ",".join(column)}
df_clean = df.groupby('Postal Code', as_index=False).aggregate(functions).reindex(columns=df.columns)
df_clean.head(10)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [3]:
df_clean.shape

(103, 3)

# Get the Coordinates for Neighborhoods

In [ ]:
latitudes, longitudes = [], []
for postal_code in df_clean['Postal Code']:
    lat_lng_coords = None
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitudes.append(lat_lng_coords[0])
    longitudes.append(lat_lng_coords[1])

df_clean['Latitude'] = latitudes
df_clean['Longitudes'] = longitudes
df_clean.head(10)

In [4]:
url = 'https://cocl.us/Geospatial_data'
df_pos = pd.read_csv(url)
df = df_clean.merge(df_pos, how='left', on='Postal Code')
df.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [6]:
df.shape

(103, 5)